In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, results as rs,scsdata

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

"""path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'
report_start = pd.to_datetime('2021-20-01 06:00:00',dayfirst=True)
report_end = pd.to_datetime('2021-20-01 18:00:00',dayfirst=True)

at = pd.read_csv(path+'at_prod.csv')
av = pd.read_csv(path+'av_prod.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
fa = pd.read_csv(path + 'fa_dec_jan.csv')

# Create sc object from Class
sc = scsdata.ScsData('scs', av, at, fa)
# Call pre-process methods
sc.pre_process_at()
sc.pre_process_av()
sc.pre_process_faults()

output, R2, num_assets = rs.run_single_model(sc_data=sc, 
            report_start=report_start, report_end=report_end, shift=0, weights=[1], speed=470, 
            picker_present=0.91, availability=0.71)"""

"path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'\nreport_start = pd.to_datetime('2021-20-01 06:00:00',dayfirst=True)\nreport_end = pd.to_datetime('2021-20-01 18:00:00',dayfirst=True)\n\nat = pd.read_csv(path+'at_prod.csv')\nav = pd.read_csv(path+'av_prod.csv')\n#fa = pd.read_csv(path + 'scs_tote_matched.csv')\nfa = pd.read_csv(path + 'fa_dec_jan.csv')\n\n# Create sc object from Class\nsc = scsdata.ScsData('scs', av, at, fa)\n# Call pre-process methods\nsc.pre_process_at()\nsc.pre_process_av()\nsc.pre_process_faults()\n\noutput, R2, num_assets = rs.run_single_model(sc_data=sc, \n            report_start=report_start, report_end=report_end, shift=0, weights=[1], speed=470, \n            picker_present=0.91, availability=0.71)"

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [18]:
at = pd.read_csv(path+'at_prod.csv')
av = pd.read_csv(path+'av_prod.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'fa_dec_jan.csv')

In [19]:
at = feat.pre_process_at(at)

Running: Function "pre_process_at" (id=2099130373888) was called 2 times


In [20]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2099130372880) was called 2 times


In [21]:
fa,unmapped = feat.pre_process_fa(scs_raw)

#fa= fa[fa['timestamp']<(fa['timestamp'].max()-pd.to_timedelta(7, unit='D'))]

Running: Function "pre_process_fa" (id=2099130375040) was called 2 times
Running: Function "load_tote_lookup" (id=2099130375472) was called 2 times
Running: Function "load_module_lookup" (id=2099127532704) was called 2 times
Running: Function "load_ID_lookup" (id=2099130373456) was called 2 times


In [22]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Running: Function "floor_shift_time_fa" (id=2099130374464) was called 2 times


In [23]:
fa_floor[fa_floor['Entry Time']>"2021-01-20 17:00"]

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type,Entry Time,End Time
238436,13303.0,114773.0,C14: C14PTT192_ external unit fault,2021-01-20 17:00:00,C14,Z,2.386294,3611,C14PTT192,PTT,Both,NaN,NaN,PTT192,14.0,4.0,19,Fault,2021-01-20 17:43:53,2021-01-20 17:43:57
238438,13314.0,132630.0,C16: C1601RCL211 211B51 sensor fault,2021-01-20 17:00:00,C16,P01,2.098612,3428,C1601RCL211,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault,2021-01-20 17:39:24,2021-01-20 17:39:27
238439,13355.0,500967.0,C37: emergency stop,2021-01-20 17:00:00,C37,Z,4.970292,0,C37,Stacker/Destacker,Blue,NaN,NaN,False,37.0,0.0,NaN,Fault,2021-01-20 17:30:53,2021-01-20 17:31:46
238440,13309.0,114773.0,C14: C14PTT192_ external unit fault,2021-01-20 17:00:00,C14,Z,2.609438,3611,C14PTT192,PTT,Both,NaN,NaN,PTT192,14.0,4.0,19,Fault,2021-01-20 17:41:03,2021-01-20 17:41:08
238441,13449.0,99481.0,C12: C1202STA082 082B22 occupancy sensor fault,2021-01-20 17:00:00,C12,P02,3.079442,3416,C1202STA082,Module Loop Grey,Grey,Module,C1202,False,12.0,3.0,14,Fault,2021-01-20 17:02:00,2021-01-20 17:02:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249754,0.0,188662.0,C23: C2301BCL177 177B51 sensor fault,2021-01-24 06:00:00,C23,P01,7.066108,3428,C2301BCL177,C23,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault,2021-01-24 06:00:00,2021-01-24 06:07:11
249755,13.0,406154.0,SCS044: sto: timeout bin from lifter to car X:...,2021-01-24 06:00:00,SCSM04,SCS044,8.220374,5105,SCS044,Carousel,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault,2021-01-24 06:00:00,2021-01-24 06:22:47
249756,1153.0,188345.0,C23: C2303BCL151 151B54 sensor fault,2021-01-24 07:00:00,C23,P03,9.188689,3428,C2303BCL151,C23,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault,2021-01-24 07:00:00,2021-01-24 08:00:00
249757,47.0,132870.0,C16: C1601STA227 227B12 occupancy sensor fault,2021-01-24 07:00:00,C16,P01,6.424950,3416,C1601STA227,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault,2021-01-24 07:00:00,2021-01-24 07:03:47


In [24]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights = [1])

#df = feat.log_totes(df) 

Running: Function "create_PTT_df" (id=2099130448048) was called 2 times
Running: Function "fault_select" (id=2099130446032) was called 37 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 37 times
Running: Function "faults_aggregate" (id=2099130446320) was called 37 times
Running: Function "weight_hours" (id=2099130447472) was called 37 times
Running: Function "av_at_select" (id=2099130446608) was called 37 times
Running: Function "aggregate_availability" (id=2099130446896) was called 37 times
Running: Function "aggregate_totes" (id=2099130447184) was called 37 times
Running: Function "merge_av_fa_at" (id=2099130447760) was called 37 times
Running: Function "fault_select" (id=2099130446032) was called 38 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 38 times
Running: Function "faults_aggregate" (id=2099130446320) was called 38 times
Running: Function "weight_hours" (id=2099130447472) was called 38 times
Running: Function "av_at_select" (i

Running: Function "fault_select" (id=2099130446032) was called 51 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 51 times
Running: Function "faults_aggregate" (id=2099130446320) was called 51 times
Running: Function "weight_hours" (id=2099130447472) was called 51 times
Running: Function "av_at_select" (id=2099130446608) was called 51 times
Running: Function "aggregate_availability" (id=2099130446896) was called 51 times
Running: Function "aggregate_totes" (id=2099130447184) was called 51 times
Running: Function "merge_av_fa_at" (id=2099130447760) was called 51 times
Running: Function "fault_select" (id=2099130446032) was called 52 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 52 times
Running: Function "faults_aggregate" (id=2099130446320) was called 52 times
Running: Function "weight_hours" (id=2099130447472) was called 52 times
Running: Function "av_at_select" (id=2099130446608) was called 52 times
Running: Function "aggregate_availa

Running: Function "fault_select" (id=2099130446032) was called 65 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 65 times
Running: Function "faults_aggregate" (id=2099130446320) was called 65 times
Running: Function "weight_hours" (id=2099130447472) was called 65 times
Running: Function "av_at_select" (id=2099130446608) was called 65 times
Running: Function "aggregate_availability" (id=2099130446896) was called 65 times
Running: Function "aggregate_totes" (id=2099130447184) was called 65 times
Running: Function "merge_av_fa_at" (id=2099130447760) was called 65 times
Running: Function "fault_select" (id=2099130446032) was called 66 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 66 times
Running: Function "faults_aggregate" (id=2099130446320) was called 66 times
Running: Function "weight_hours" (id=2099130447472) was called 66 times
Running: Function "av_at_select" (id=2099130446608) was called 66 times
Running: Function "aggregate_availa

In [17]:
fa_PTT['PTT011']

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,PLCN,Quadrant,MODULE,Alert Type,Entry Time,End Time,Pick Station
77271,191528.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 00:00:00,C05,Z,4.828641,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 00:32:00,2020-11-20 00:32:46,PTT011
77272,190344.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 03:00:00,C05,Z,2.791759,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 03:46:00,2020-11-20 03:46:06,PTT011
77273,190141.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 04:00:00,C05,Z,2.609438,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 04:25:00,2020-11-20 04:25:05,PTT011
77274,189201.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 07:00:00,C05,Z,4.663562,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 07:46:00,2020-11-20 07:46:39,PTT011
77275,189058.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 08:00:00,C05,Z,2.609438,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 08:18:00,2020-11-20 08:18:05,PTT011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249664,19345.0,123351.0,C15: C1501ACH087 087QM11 motor circuit breaker...,2021-01-19 06:00:00,C15,P01,6.605802,3430.0,C1501ACH087,C15,Blue,Outside,C1501,15.0,0.0,NaN,Fault,2021-01-19 06:26:51,2021-01-19 06:31:23,False
249692,15091.0,122423.0,C15: C1502STA028 028B21 occupancy sensor fault,2021-01-20 09:00:00,C15,P02,2.609438,3416.0,C1502STA028,C15,Blue,Outside,C1502,15.0,0.0,NaN,Fault,2021-01-20 09:55:53,2021-01-20 09:55:58,False
249704,13896.0,123189.0,C15: C1502RDC077 077B11 occupancy sensor fault,2021-01-20 14:00:00,C15,P02,5.094345,3416.0,C1502RDC077,C15,Blue,Outside,C1502,15.0,0.0,NaN,Fault,2021-01-20 14:59:00,2021-01-20 15:00:00,False
249705,13896.0,123189.0,C15: C1502RDC077 077B11 occupancy sensor fault,2021-01-20 15:00:00,C15,P02,7.188264,3416.0,C1502RDC077,C15,Blue,Outside,C1502,15.0,0.0,NaN,Fault,2021-01-20 15:00:00,2021-01-20 15:08:07,False


In [ ]:
df_week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]

In [ ]:
X,y = md.gen_feat_var(df,target = 'Availability', features = ['Faults','Totes'])


In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': 42})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

In [ ]:
R2_OOS

# Plotting

In [ ]:
Output[Output['AREA'] == 'PTT']['ID'].value_counts()

In [ ]:
Total = Output[['ID','SINGLES']].groupby('ID').sum().reset_index()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(y='SINGLES',x = 'ID',data = Total.sort_values(by='SINGLES',ascending=False).head(10),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
PTT081 = Output[['Asset Code','PTT081']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(y='PTT081',x = 'Asset Code',data = PTT081.sort_values(by='PTT081',ascending=False).head(5),color = 'r')
plt.ylabel('Lost Singles')
plt.title('PTT081 Lost Singles Last 12 Hours')

In [ ]:
Total_colour = Output[['BLUEGREY','SINGLES']].groupby('BLUEGREY').sum().reset_index()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(y='SINGLES',x = 'BLUEGREY',data = Total_colour.sort_values(by='SINGLES',ascending=False),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
Total_colour

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
sns.lmplot(x='TOTES', y='Availability', data=df_week, line_kws={"color": "red"})

In [ ]:
ax = sns.regplot(x="TOTES", y="Availability", data=df_week,
x_estimator=np.mean, logx=True, line_kws={"color": "red"})
plt.ylim(0,1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df_week, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
Shift = [0,0,0,10,10,10,20,20,20]
Weights = [[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1]]

In [ ]:
Weights[8]

In [ ]:
y

In [ ]:
at

In [ ]:
fa_floor

In [ ]:
df.drop(['Availability','Blue Tote Loss','Grey Tote Loss','timestamp'],axis=1)

In [ ]:
Outputs = {0.5:'Number One',0.4:'Number Two',-1.4:'Number 3'}

In [ ]:
 Output = Outputs[max(k for k, v in Outputs.items())]

In [ ]:
Outputs[0.456]='Numbers 4'

In [ ]:
Outputs

In [ ]:
print('Selected R2:', max(k for k, v in Outputs.items()))

In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=10)

df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights=[0.7,0.2,0.1])
df = feat.log_totes(df) 
df_2week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]
X,y = md.gen_feat_var(df_2week,target = 'Availability', features = ['Totes','Faults'])
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                    'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

Output = rs.create_output(fa_PTT,Coeff,end_time,speed = speed, picker_present = picker_present, availability = availability)